# import packages 

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
from tensorflow.python.framework import ops
ops.reset_default_graph()
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


# open json file

In [13]:
with open("intents.json") as file:
    data = json.load(file)

# Pre processing 

## tokenization

In [14]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

## Stemming

In [15]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]

# sort 
words = sorted(list(set(words)))
labels = sorted(labels)

## spliting Data 

In [16]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

### bag of words 

In [17]:
# one hot encoding
for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

### prepared data 

In [18]:
training = numpy.array(training)
output = numpy.array(output)

with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

# Training

## DNN model 

In [8]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## fiting the model

In [9]:
model.fit(training, output, n_epoch=1500, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 14999  | total loss: 0.01809 | time: 0.033s
| Adam | epoch: 1500 | loss: 0.01809 - acc: 0.9947 -- iter: 72/76
Training Step: 15000  | total loss: 0.01686 | time: 0.038s
| Adam | epoch: 1500 | loss: 0.01686 - acc: 0.9953 -- iter: 76/76
--
INFO:tensorflow:C:\Users\kamal\Desktop\cv_chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


# Testing

## bag of words for the input 

In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

## predicting the answer 

In [11]:
def chat():
    print("Start talking with the bot (To stop, type quit or any expression to say bye)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print("KHALID : ",random.choice(responses),"\n")
        if tag == "goodbye":
            break

## run  

In [12]:
chat()

Start talking with the bot (To stop, type quit or any expression to say bye)!
You: hi
KHALID :  Good to see you again! 

You: how are you
KHALID :  Thank you for asking, I'm fine, hope you're doing well too ! 

You: akills
KHALID :  Sorry I didn't get it, please try another question ! 

You: skills
KHALID :  MY TECHNICAL SKILLS --- ***** --- *** Programming Languages *** : PYTHON, R, JAVA, C, JAVASCRIPT, PHP. *** Data Science Skills *** : Machine Learning, Deep learning, Artificial Intelligence, Data Manipulation, DataVisualization, Business Intelligence, Data mining, Web scraping, *** Machine Learning and Deep Learning *** : Tensorflow, sklearn, NLP, ** models ** : SVM, KNN, Decesion Tree, Random Forest, K-means, PCA, RNN, CNN... *** Data processing *** : numpy, pandas. *** Data visualization *** : matplotlib, tableau. *** Web Development *** : Spring Boot, Angular, Laravel, Mysql, Nosql. 

You: projects
KHALID :  MY ACADEMIC PROJECTS --- ***** --- *** Chatbot application (05/2021 - 0